In [1]:
# 1. Import pustaka
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

In [5]:
# Adjust target to binary classification: 0 for versicolor, 1 for virginica
y = iris.target[iris.target != 0] - 1  # Exclude class 0 (setosa) and adjust target

# One-hot encoding is not needed for binary classification
# Split data train-test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Update the model output layer and loss function for binary classification
def build_mlp(layers_config, lr=0.001):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X.shape[1],)))
    for units in layers_config:
        model.add(layers.Dense(units, activation="relu"))
    model.add(layers.Dense(1, activation="sigmoid"))  # Binary classification output
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="binary_crossentropy",  # Binary crossentropy loss
        metrics=["accuracy"],
    )
    return model

# Update training and evaluation loop
histories = {}
results = {}

for name, config in architectures.items():
    print(f"\n🚀 Training {name} with hidden config {config}")
    model = build_mlp(config, lr=0.001)
    history = model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=200,
        batch_size=16,
        verbose=0,
        callbacks=callbacks
    )
    histories[name] = history
    
    # Evaluation
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    y_pred = (model.predict(X_test) > 0.5).astype(int).flatten()
    
    results[name] = {
        "Test Accuracy": test_acc,
        "Classification Report": classification_report(y_test, y_pred, digits=4)
    }

# Print results
for name, res in results.items():
    print(f"\n===== {name} =====")
    print("✅ Test Accuracy:", res["Test Accuracy"])
    print(res["Classification Report"])



🚀 Training MLP_1layer_8 with hidden config [8]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step

🚀 Training MLP_2layer_16 with hidden config [16, 16]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step

🚀 Training MLP_3layer_32 with hidden config [32, 32, 32]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step

🚀 Training MLP_4layer_64 with hidden config [64, 64, 64, 64]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step

===== MLP_1layer_8 =====
✅ Test Accuracy: 0.800000011920929
              precision    recall  f1-score   support

           0     0.7500    0.9000    0.8182        10
           1     0.8750    0.7000    0.7778        10

    accuracy                         0.8000        20
   macro avg     0.8125    0.8000    0.7980        20
weighted avg     0.8125    0.8000    0.7980        20


===== MLP_2layer_16 =====
✅ Test Accuracy: 0.800000011920929
              precision    recall  f1-score   support

           0     0.7500    0.9000    0.8182        10
           1     0.8750    0.7000    0.7778        10

    accuracy

In [4]:
import plotly.graph_objects as go

# Data Asli (dalam mikrosekon / us)
cases = ['A', 'B', 'C', 'D', 'E']
esp32_us = [1512.71, 1479.37, 1716.26, 1769.97, 8906.33]
rp2350_us = [1240.63, 1240.77, 1447.00, 1577.89, 7215.46]

# KONVERSI KE MILLISECOND (ms)
# Membagi nilai dengan 1000
esp32_ms = [x / 1000 for x in esp32_us]
rp2350_ms = [x / 1000 for x in rp2350_us]

# Membuat Figure
fig = go.Figure()

# Menambahkan Bar untuk ESP32 (Merah - Lambat)
fig.add_trace(go.Bar(
    y=cases,
    x=esp32_ms,
    name='ESP32 (INT8)',
    orientation='h',
    marker=dict(color='#EF553B', line=dict(width=0)),
    text=esp32_ms,
    textposition='auto',
    texttemplate='%{text:.2f}', # Menampilkan 2 angka di belakang koma
    hovertemplate='<b>Waktu:</b> %{x:.3f} ms<extra></extra>'
))

# Menambahkan Bar untuk RP2350 (Hijau - Cepat)
# Menghitung Speedup untuk tooltip
speedup = [(e - r) / e * 100 for e, r in zip(esp32_ms, rp2350_ms)]
custom_data = [f'{s:.1f}% Lebih Cepat' for s in speedup]

fig.add_trace(go.Bar(
    y=cases,
    x=rp2350_ms,
    name='RP2350 (INT8)',
    orientation='h',
    marker=dict(color='#00CC96', line=dict(width=0)),
    text=rp2350_ms,
    textposition='auto',
    texttemplate='%{text:.2f}', # Menampilkan 2 angka di belakang koma
    hovertext=custom_data,
    hovertemplate='<b>Waktu:</b> %{x:.3f} ms<br><b>Performa:</b> %{hovertext}<extra></extra>'
))

# Update Layout
fig.update_layout(
    title='<b>Perbandingan Latensi Inferensi INT8 (ms): ESP32 vs RP2350</b>',
    xaxis_title='Inference Time (ms)', # Label sumbu sudah diganti ke ms
    yaxis_title='Case (Model Scenarios)',
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    plot_bgcolor='white',
    width=900,
    height=600,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    font=dict(family="Arial", size=12, color="#2c3e50")
)

# Grid Vertikal
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#ecf0f1')
fig.update_yaxes(showgrid=False)

fig.show()